In [1]:
import glob
import os


# Reminder to install s3fs to read files from aws
import s3fs

# import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import xarray as xr

import seaborn as sns

# import scanpy.api as sc

# import holoviews as hv
# hv.extension('bokeh')
# hv.archive.auto()

%matplotlib inline

In [35]:
prefix = 's3://olgabot-maca/facs/sourmash_compare_no_track_abundance/'
txt = 'aws_s3_ls_tabula_muris_compare_no_track_abundance.txt'

In [36]:
! aws s3 ls $prefix > $txt

In [54]:
aws_s3_ls = pd.read_table(txt, delim_whitespace=True, header=None, 
                          names=['date', 'time', 'bytes', 'basename'])
print(aws_s3_ls.shape)
aws_s3_ls.head()

(150, 4)


,date,time,bytes,basename
0,2018-09-23,16:46:50,19045449,trim=false_scaled=1000_dna_ksize=12.csv
1,2018-09-18,01:32:22,19873413,trim=false_scaled=1000_dna_ksize=15.csv
2,2018-09-23,16:44:28,10580690,trim=false_scaled=1000_dna_ksize=33.csv
3,2018-09-17,22:08:11,20376123,trim=false_scaled=1000_dna_ksize=42.csv
4,2018-09-23,16:46:49,20396041,trim=false_scaled=1000_dna_ksize=45.csv


In [38]:
from tqdm import tqdm

In [39]:

def extract_cell_metadata(columns):
    cell_metadata = pd.Series(columns).str.split('|', expand=True)
#     cell_metadata.index = columns
    cell_metadata.columns = ['cell_id', 'tissue', 
                             'subtissue', 'cell_ontology_class', 
                             'free_annotation']
    cell_metadata = cell_metadata.set_index('cell_id')
    cell_metadata.index.name = 'cell_id'
    cell_metadata = cell_metadata.applymap(lambda x: x.split(':')[-1])
#     print('\tcell_metadata.shape', cell_metadata.shape)
#     cell_metadata = cell_metadata.sort_index()
#     cell_metadata.head()
    return cell_metadata



dfs = {}

metadatas = []

def to_key_value_pair(attribute):
    if len(attribute) > 1:
        try:
            return attribute[0], int(attribute[1])
        except ValueError:
            return attribute[0], attribute[1] 
    else:
        return 'comparison_sequence', attribute[0]



def extract_experiment_metadata(basename):
    key = basename.split('.csv')[0]
    split = key.split('_')
    attributes = [x.split('=') for x in split]
    attributes = dict(to_key_value_pair(x) for x in attributes)
    return key, attributes


experiment_metadatas = {}

cell_metadata = pd.DataFrame()

max_cells = 1000
tidy_dfs = []

for basename in tqdm(aws_s3_ls['basename']):
    csv = f'{prefix}{basename}'
    df = pd.read_csv(csv)
#     aprint(f'{basename}\tdf.shape: {df.shape}')
    
    key, attributes = extract_experiment_metadata(basename)
    experiment_metadatas[key] = attributes

    metadata = extract_cell_metadata(df.columns)
    
    mask = np.triu(np.ones(df.shape)).astype(np.bool)

    df.index = metadata.index.copy()
    df.columns = metadata.index.copy()
    
    df = df.where(mask)
    df.index.name = 'cell_id'
    df.columns.name = 'other_cell'
    tidy = df.stack().reset_index()
    tidy['experiment'] = key
#     tidy = tidy.rename(columns={'level_0': 'cell_id', 'level_1': 'other_cell'})
        
    tidy_dfs.append(tidy)

    # Add new cells to metadata
    if len(df.index.difference(cell_metadata.index)) > 0:
        cell_metadata = cell_metadata.append(metadata)
        cell_metadata = cell_metadata.loc[~cell_metadata.index.duplicated()]
        print(f'\tcell_metadata.shape, {cell_metadata.shape}')



  1%|          | 1/150 [00:01<03:46,  1.52s/it]

	cell_metadata.shape, (997, 4)


  5%|▌         | 8/150 [00:16<04:50,  2.04s/it]

	cell_metadata.shape, (1000, 4)


100%|██████████| 150/150 [06:00<00:00,  2.40s/it]


In [40]:
compare_results = pd.concat(tidy_dfs, ignore_index=True)
compare_results = compare_results.rename(columns={0: 'similarity'})
print(compare_results.shape)
compare_results.head()

(59524382, 4)


,cell_id,other_cell,similarity,experiment
0,B22-MAA000871-3_11_M-1-1,B22-MAA000871-3_11_M-1-1,1.000000,trim=false_scaled=1000_dna_ksize=12
1,B22-MAA000871-3_11_M-1-1,L11-MAA000586-3_8_M-1-1,0.398906,trim=false_scaled=1000_dna_ksize=12
2,B22-MAA000871-3_11_M-1-1,O20-MAA001632-3_56_F-1-1,0.407873,trim=false_scaled=1000_dna_ksize=12
3,B22-MAA000871-3_11_M-1-1,D2-MAA000452-3_8_M-1-1,0.386677,trim=false_scaled=1000_dna_ksize=12
4,B22-MAA000871-3_11_M-1-1,P1-D042473-3_10_M-1-1,0.384180,trim=false_scaled=1000_dna_ksize=12


In [41]:
compare_results.tail()

,cell_id,other_cell,similarity,experiment
59524377,J9-MAA001892-3_38_F-1-1,O2-MAA001454-3_38_F-1-1,0.053038,trim=true_scaled=900_dna_ksize=21
59524378,J9-MAA001892-3_38_F-1-1,F6-MAA000871-3_11_M-1-1,0.053920,trim=true_scaled=900_dna_ksize=21
59524379,O2-MAA001454-3_38_F-1-1,O2-MAA001454-3_38_F-1-1,1.000000,trim=true_scaled=900_dna_ksize=21
59524380,O2-MAA001454-3_38_F-1-1,F6-MAA000871-3_11_M-1-1,0.052045,trim=true_scaled=900_dna_ksize=21
59524381,F6-MAA000871-3_11_M-1-1,F6-MAA000871-3_11_M-1-1,1.000000,trim=true_scaled=900_dna_ksize=21


In [42]:
def combine_cell_ontology_free_annotation(row):
    if pd.notnull(row['free_annotation']):
        return '{cell_ontology_class} ({free_annotation})'.format(**row)
    else:
        return row['cell_ontology_class']

In [43]:
# cell_metadata = pd.concat(metadatas)
# print(cell_metadata.shape)
# cell_metadata = cell_metadata.loc[~cell_metadata.index.duplicated()]
cell_metadata = cell_metadata.sort_index()
cell_metadata['cell_ontology_free_annotation'] = cell_metadata.apply(combine_cell_ontology_free_annotation, axis=1)
print(cell_metadata.shape)
cell_metadata.head()

(1000, 5)


,tissue,subtissue,cell_ontology_class,free_annotation,cell_ontology_free_annotation
cell_id,,,,,
A1-B002427-3_39_F-1-1,Heart,LA,myofibroblast_cell,NA,myofibroblast_cell (NA)
A1-D041914-3_8_M-1-1,Bladder,NA,bladder_cell,Bladder_mesenchymal_cell,bladder_cell (Bladder_mesenchymal_cell)
A1-MAA000496-3_10_M-1-1,Trachea,NA,mesenchymal_cell,NA,mesenchymal_cell (NA)
A1-MAA000549-3_8_M-1-1,Skin,Anagen,epidermal_cell,Intermediate_IFE,epidermal_cell (Intermediate_IFE)
A1-MAA001869-3_38_F-1-1,Large_Intestine,Proximal,epithelial_cell_of_large_intestine,Lgr5-_amplifying_undifferentiated_cell,epithelial_cell_of_large_intestine (Lgr5-_ampl...


In [44]:
experiment_metadata = pd.DataFrame(experiment_metadatas).T
experiment_metadata['trim'] = experiment_metadata['trim'].map(lambda x: x == 'true')
experiment_metadata['ksize'] = experiment_metadata['ksize'].astype(int)
experiment_metadata['scaled'] = experiment_metadata['scaled'].astype(int)
print(experiment_metadata.shape)
experiment_metadata.head()

(150, 4)


,comparison_sequence,ksize,scaled,trim
trim=false_scaled=1000_dna_ksize=12,dna,12,1000,False
trim=false_scaled=1000_dna_ksize=15,dna,15,1000,False
trim=false_scaled=1000_dna_ksize=33,dna,33,1000,False
trim=false_scaled=1000_dna_ksize=42,dna,42,1000,False
trim=false_scaled=1000_dna_ksize=45,dna,45,1000,False


In [45]:
sorted(experiment_metadata.scaled.unique())

[200,
 300,
 400,
 500,
 600,
 700,
 800,
 900,
 1000,
 1100,
 1200,
 1300,
 1400,
 1500,
 1600,
 1700,
 1800,
 1900,
 2000,
 2500,
 3000,
 3500,
 4000,
 4500,
 5000]

In [46]:
csv_prefix = 's3://olgabot-maca/facs/sourmash_compare_no_track_abundance_combined/'

In [47]:
# with fs.open(f'{csv_prefix}experiment-metadata.csv', 'w') as f:
#     experiment_metadata.to_csv(f)
# # %time experiment_metadata.to_csv(f'{csv_prefix}experiment-metadata.csv')

In [48]:
# pd.read_csv(f'{csv_prefix}experiment-metadata.csv').head()

In [49]:
%%file s3_utils.py
import s3fs

def write_s3(df, filename, fmt='csv', **kwargs):
    fs = s3fs.S3FileSystem(anon=False)
    if fmt == 'csv':
        # csv is a text format
        with fs.open(filename, 'w') as f:
            return df.to_csv(f, **kwargs)
    elif fmt == 'parquet':
        # Parquet is a binary format and needs the "b" flag
        with fs.open(filename, 'wb') as f:
            return df.to_parquet(f, **kwargs)
        

Overwriting s3_utils.py


In [50]:
import s3fs

fs = s3fs.S3FileSystem(anon=False)

with fs.open(f'{csv_prefix}similarity.parquet', 'wb') as f:
    %time compare_results.to_parquet(f)

CPU times: user 41.1 s, sys: 36.1 s, total: 1min 17s
Wall time: 1min 23s


In [51]:
from s3_utils import write_s3

%time write_s3(cell_metadata, f'{csv_prefix}cell-metadata.csv', 'csv')
%time write_s3(experiment_metadata, f'{csv_prefix}experiment-metadata.csv', 'csv')


CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 80.8 ms
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 69.2 ms


In [52]:
%time write_s3(compare_results, f'{csv_prefix}similarity.csv', 'csv')


CPU times: user 5min 6s, sys: 6.62 s, total: 5min 13s
Wall time: 10min 55s


In [53]:
! aws s3 ls --human-readable $csv_prefix

2018-09-23 16:03:11  334.5 MiB -similarity.parquet
2018-09-23 17:36:09  102.3 KiB cell-metadata.csv
2018-09-23 17:36:09    7.7 KiB experiment-metadata.csv
2018-09-23 17:36:09    6.1 GiB similarity.csv
2018-09-23 17:35:32  649.8 MiB similarity.parquet


In [ ]:
# write_s3(compare_results, f'{csv_prefix}-similarity.parquet', 'parquet')

In [ ]:
# with fs.open(f'{csv_prefix}cell-metadata.csv', 'w') as f:
#     cell_metadata.to_csv(f)

In [ ]:
# with fs.open(f'{csv_prefix}similarity.csv', 'w') as f:
#     %time compare_results.to_csv(f, index=False)

In [1]:
# ls -lha $csv_prefix*

-rwxrwxr-x 1 olga olga 4.3M Jul 16 09:03 000_tabula-muris_reflow_fastqs.ipynb*
-rwxrwxr-x 1 olga olga 170K Aug  7 13:01 004_ashley_cancer_fastq.ipynb*
-rwxrwxr-x 1 olga olga 116K Jul 16 09:03 005_why_did_some_reflow_runs_fail.ipynb*
-rw-rw-r-- 1 olga olga 138K Jul 27 12:27 006_tabula-muris_reflow_fastqs_v5.ipynb
-rw-rw-r-- 1 olga olga  67K Jul 27 12:27 006_tabula-muris_reflow_fastqs_v6.ipynb
-rwxrwxr-x 1 olga olga  43K Sep 19 10:59 007_reflow_runbatch_compare.ipynb*
-rw-rw-r-- 1 olga olga  17K Sep 19 09:25 008_ashley_cancer_compute_v2.ipynb
-rw-rw-r-- 1 olga olga  15K Sep 20 12:21 009_ashley_cancer_compare.ipynb
-rw-rw-r-- 1 olga olga 1.3M Aug  2 14:49 010_sourmash_compare_analysis.ipynb
-rw-rw-r-- 1 olga olga 303K Aug 14 12:15 011_kmers_in_single_bladder_cell_vs_gencode_transcripts.ipynb
-rw-rw-r-- 1 olga olga 6.0M Aug 14 12:15 012_sourmash_compare_csv_scale1200_k15.ipynb
-rw-rw-r-- 1 olga olga 1.6M Sep 21 15:08 013_read_all_tabula_muris_sourmash_compare_csvs.ipynb
-rw-rw-r-- 1 olga o